In [124]:
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, recall_score, confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt#
from xgboost import XGBClassifier

import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [125]:
# Data was cleaned in excel, data quality was good. There were no missing values or duplicates. 

In [126]:
df = pd.read_excel('insurance_data_sample.xlsx', sheet_name='final_sheet')

# Create income brackets
income_bins = [0, 20000, 40000, 60000, 80000, np.inf]
labels = ['Low', 'Lower-Middle', 'Middle', 'Upper-Middle', 'High']
df['Income_Bracket'] = pd.cut(df['Annual Income'], bins=income_bins, labels=labels)

# Create a summary statistics DataFrame

summary_stats = pd.DataFrame()
summary_stats['Mode'] = df.mode().iloc[0]
summary_stats['Mode Count'] = df.apply(lambda x: x.value_counts().iloc[0])
summary_stats['Unique Values'] = df.nunique()
summary_stats['Unique Value Counts'] = df.apply(lambda x: x.nunique())
summary_stats['Missing Values'] = df.isnull().sum()
summary_stats['Data Type'] = df.dtypes

display(summary_stats)


,Mode,Mode Count,Unique Values,Unique Value Counts,Missing Values,Data Type
Quarter,4,7520,4,4,0,int64
Year,2023,13150,2,2,0,int64
Gender,Male,18634,2,2,0,object
Annual Income,13500,5234,2500,2500,0,int64
Dealer_Name,Progressive Shippers Cooperative Association No,1306,28,28,0,object
Company,Chevrolet,1819,29,29,0,object
Model,Diamante,418,152,152,0,object
Engine,Double Overhead Camshaft,12394,2,2,0,object
Transmission,Auto,12394,2,2,0,object
Color,Pale White,11205,3,3,0,object


# Predicting Insurance Claims

Help us identify customers who are more likely to claim insurance. These customers can be charged with higher prices.

In [127]:
# # Drop the columns 'Claim amount' and 'Claim'
# X = df.drop(['Claim amount', 'Claim'], axis=1)
# y = df['Claim']

# # Select only numeric columns
# numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
# X = X[numeric_cols]

# # Handle class imbalance
# ros = RandomOverSampler(random_state=42)
# X_resampled, y_resampled = ros.fit_resample(X, y)

# # Split data into train, validation, and test sets
# X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.4, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# # Calculate class weights
# class_weights = {0: 1, 1: len(y_train) / y_train.sum()}

# # Define the XGBoost classifier
# xgb_classifier = XGBClassifier(objective='binary:logistic', eval_metric='logloss', class_weight=class_weights, use_label_encoder=False)

# # Define the parameter grid
# param_grid = {
#     'n_estimators': [400, 450, 500],
#     'learning_rate': [0.15, 0.17, 0.2],
#     'max_depth': [12, 15, 18],
#     'subsample': [0.8, 1.0],
#     'colsample_bytree': [0.8, 1.0]
# }

# # Define the custom scoring function
# recall_scorer = make_scorer(recall_score, pos_label=1)

# # Initialize GridSearchCV
# grid_cv = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, scoring=recall_scorer, cv=3, verbose=1, n_jobs=-1)

# # Fit GridSearchCV
# grid_cv.fit(X_train, y_train)

# # Print the best parameters and the best recall score
# print("Best parameters found: ", grid_cv.best_params_)
# print("Best recall for class '1': ", grid_cv.best_score_)

# # Evaluate the best model on the validation set
# best_model = grid_cv.best_estimator_
# y_val_pred = best_model.predict(X_val)
# print("\nValidation Set Performance with Best Model:")
# print("Confusion Matrix:")
# print(confusion_matrix(y_val, y_val_pred))
# print("\nClassification Report:")
# print(classification_report(y_val, y_val_pred))

# # Evaluate the best model on the test set
# y_test_pred = best_model.predict(X_test)
# print("\nTest Set Performance with Best Model:")
# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_test_pred))
# print("\nClassification Report:")
# print(classification_report(y_test, y_test_pred))



In [128]:
# Drop the columns 'Claim amount' and 'Claim'
X = df.drop(['Claim amount', 'Claim'], axis=1)
y = df['Claim']

# Select only numeric columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
X = X[numeric_cols]

# Handle class imbalance
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Split data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Calculate class weights
class_weights = {0: 1, 1: len(y_train) / y_train.sum()}

# Define the XGBoost classifier with the best parameters
xgb_classifier = XGBClassifier(objective='binary:logistic', eval_metric='logloss',
                                n_estimators=500, learning_rate=0.2, max_depth=15,
                                subsample=0.8, colsample_bytree=0.8, class_weight=class_weights,
                                use_label_encoder=False)

# Fit the model
xgb_classifier.fit(X_train, y_train)

# Evaluate the model on the validation set
y_val_pred = xgb_classifier.predict(X_val)
print("\nValidation Set Performance with Best Model:")
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred))

# Evaluate the model on the test set
y_test_pred = xgb_classifier.predict(X_test)
print("\nTest Set Performance with Best Model:")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))


C:\Users\taha_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:160: UserWarning: [18:11:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)



Validation Set Performance with Best Model:
Confusion Matrix:
[[3741  490]
 [  12 4289]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.88      0.94      4231
           1       0.90      1.00      0.94      4301

    accuracy                           0.94      8532
   macro avg       0.95      0.94      0.94      8532
weighted avg       0.95      0.94      0.94      8532


Test Set Performance with Best Model:
Confusion Matrix:
[[3802  509]
 [   6 4215]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.88      0.94      4311
           1       0.89      1.00      0.94      4221

    accuracy                           0.94      8532
   macro avg       0.95      0.94      0.94      8532
weighted avg       0.95      0.94      0.94      8532



To address the class imbalance problem in our dataset, we utilized `RandomOverSampler` to balance the number of samples in each class. This was essential as our data had significantly more instances of non-claims compared to claims. 

We chose XGBoost, a powerful gradient boosting algorithm that excels in handling tabular data, for our classification task. To find the optimal hyperparameters for XGBoost, we employed GridSearchCV, focusing specifically on maximizing the recall of class '1' (insurance claims). This approach ensured that our model was particularly effective at predicting the minority class, which is crucial in our insurance context.

Our dataset was divided into training, validation, and test sets to rigorously evaluate model performance. The best model identified through GridSearchCV had the following parameters: `{'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 15, 'n_estimators': 500, 'subsample': 0.8}`, achieving a recall of 0.9139 for class '1'.

### Validation Set Performance with Best Model:
- **Confusion Matrix**:
  ```
  [[3741  490]
   [ 12 4289]]
  ```
- **Classification Report**:
  ```
                precision    recall  f1-score   support

             0       1.00      0.88      0.94      4231
             1       0.90      1.00      0.94      4301

      accuracy                           0.94      8532
     macro avg       0.95      0.94      0.94      8532
  weighted avg       0.95      0.94      0.94      8532
  ```

### Test Set Performance with Best Model:
- **Confusion Matrix**:
  ```
  [[3802  509]
   [  6  4215]]
  ```
- **Classification Report**:
  ```
                precision    recall  f1-score   support

             0       1.00      0.88      0.94      4311
             1       0.89      1.00      0.94      4221

      accuracy                           0.89      8532
     macro avg       0.95      0.94      0.89      8532
  weighted avg       0.95      0.94      0.89      8532
  ```


In [129]:
# Select a chunk of data (let's say the first 100 rows)
X_chunk = df.drop(['Claim amount', 'Claim'], axis=1).head(20000)
y_chunk = df['Claim'].head(20000)

# Select only numeric columns
numeric_cols = X_chunk.select_dtypes(include=['int64', 'float64']).columns
X_chunk = X_chunk[numeric_cols]

# Predict using the best model
y_chunk_pred = best_model.predict(X_chunk)

# Evaluate the model performance on the chunk
print("Chunk Data Performance with Best Model:")
print("Confusion Matrix:")
print(confusion_matrix(y_chunk, y_chunk_pred))
print("\nClassification Report:")
print(classification_report(y_chunk, y_chunk_pred))


Chunk Data Performance with Best Model:
Confusion Matrix:
[[17088   913]
 [    3  1996]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97     18001
           1       0.69      1.00      0.81      1999

    accuracy                           0.95     20000
   macro avg       0.84      0.97      0.89     20000
weighted avg       0.97      0.95      0.96     20000



It even performed well on random sample from data, indicating our model has now generalized well!

# Forecasting Insurance Purchase and Claims

 - Predicted Claims and Insurance Purchase for year 2024

In [130]:
yearly_claims = pd.read_excel('insurance_data_sample.xlsx',sheet_name='insurance_overtime')


# Extract features and target variable
X = yearly_claims[['Year', 'Quarter']]
y_claimed = yearly_claims['insurance_claimed']
y_paid = yearly_claims['insurance_paid']

# Initialize linear regression models
model_claimed = LinearRegression()
model_paid = LinearRegression()

# Fit the models
model_claimed.fit(X, y_claimed)
model_paid.fit(X, y_paid)


# Predict insurance claimed and insurance paid for each quarter of the next year
next_year = 2024
quarters = [1, 2, 3, 4]

predicted_claimed = model_claimed.predict(np.array([[next_year, q] for q in quarters]))
predicted_paid = model_paid.predict(np.array([[next_year, q] for q in quarters]))

# Create a DataFrame to display the results
next_year_predictions = pd.DataFrame({
    'Year': next_year,
    'Quarter': quarters,
    'Predicted Insurance Claims': predicted_claimed,
    'Predicted Insurance Purchase': predicted_paid
})


next_year_predictions

C:\Users\taha_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\taha_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


,Year,Quarter,Predicted Insurance Claims,Predicted Insurance Purchase
0,2024,1,857110.325,5072805.4
1,2024,2,1025032.775,5981497.8
2,2024,3,1192955.225,6890190.2
3,2024,4,1360877.675,7798882.6


# Ideal Customer Profile

In [131]:
# Select relevant features for clustering
features = ['Gender', 'Annual Income', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color', 'Price ($)', 'Body Style', 'Amount_paid_for_insurance', 'City', 'Income_Bracket']

# Convert 'Income_Bracket' to categorical if not already
df['Income_Bracket'] = df['Income_Bracket'].astype('category')

# Preprocess the data
numeric_features = ['Annual Income', 'Price ($)', 'Amount_paid_for_insurance']
categorical_features = ['Gender', 'Dealer_Name', 'Company', 'Model', 'Engine', 'Transmission', 'Color', 'Body Style', 'City', 'Income_Bracket']

# Create preprocessing pipelines for both numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Apply preprocessing
X_preprocessed = preprocessor.fit_transform(df[features])

# Convert the preprocessed data back to a dense array for better interpretability
X_preprocessed_array = X_preprocessed.toarray()

# Convert the preprocessed data array to a DataFrame
preprocessed_feature_names = numeric_features + list(preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out())
X_preprocessed_df = pd.DataFrame(X_preprocessed_array, columns=preprocessed_feature_names)

# Number of clusters
n_clusters = 4

# Initialize KMeans clustering model
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit the model to the preprocessed data
kmeans.fit(X_preprocessed_df)

# Get cluster labels for each data point
cluster_labels = kmeans.labels_

# Assign cluster labels to the original DataFrame
df['Cluster'] = cluster_labels+1

# Display the DataFrame with cluster labels
display(df.head())



,Quarter,Year,Gender,Annual Income,Dealer_Name,Company,Model,Engine,Transmission,Color,Price ($),Body Style,Amount_paid_for_insurance,Claim amount,City,Claim,Income_Bracket,Cluster
0,1,2022,Female,671500,Scrivener Performance Engineering,Toyota,Sienna,Overhead Camshaft,Manual,Pale White,39501,Passenger,2182,0,Ventspils,0,High,1
1,1,2022,Male,765000,Star Enterprises Inc,Lincoln,Navigator,Double Overhead Camshaft,Auto,Black,19001,Sedan,1280,0,Liepaja,0,High,2
2,1,2022,Male,900000,Suburban Ford,Nissan,Pathfinder,Double Overhead Camshaft,Auto,Pale White,21001,SUV,1202,0,Ventspils,0,High,2
3,1,2022,Female,820000,Tri-State Mack Inc,Mercury,Villager,Overhead Camshaft,Manual,Pale White,16000,Passenger,832,0,Ventspils,0,High,3
4,1,2022,Female,650000,U-Haul CO,Chrysler,Concorde,Double Overhead Camshaft,Auto,Black,12000,SUV,1046,0,Riga,0,High,2


In [136]:
# Example input data
example_data = [
    ['Female', 671500, 'Toyota', 'Sienna', 'Pale White', 39501, 'Ventspils', 'Sienna', 2182, 'Corolla', 'Overhead Camshaft', 'Automatic', 'High'],
]

# Convert input data to DataFrame
example_df = pd.DataFrame(example_data, columns=['Gender', 'Annual Income', 'Company', 'Body Style', 'Color', 'Price ($)', 'City', 'Dealer_Name', 'Amount_paid_for_insurance', 'Model', 'Engine', 'Transmission', 'Income_Bracket'])

# Apply the same preprocessing steps used for the original data
X_preprocessed_example = preprocessor.transform(example_df)

# Use the trained KMeans model to predict clusters for the example data
example_cluster_labels = kmeans.predict(X_preprocessed_example)

# Display the cluster labels for each example data point
for i, label in enumerate(example_cluster_labels):
    print(f"Example belongs to cluster {label+1}")



Example belongs to cluster 1


C:\Users\taha_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


In [140]:
# Define the important features related to customer demographics
important_features = ['Annual Income', 'Price ($)', 'Amount_paid_for_insurance']

# Define the summary statistics to compute
summary_statistics = {
    'mean': 'mean',
    'median': 'median',
    'min': 'min',
    'max': 'max',
    'std': 'std',
    'mode': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else None  # Custom function to compute mode
}

# Perform the specified summary statistics for each feature and each cluster
cluster_summaries = {}
for feature in important_features:
    feature_summary = df.groupby('Cluster')[feature].agg(**summary_statistics)
    cluster_summaries[feature] = feature_summary

# Display the summary statistics for each feature and each cluster
for feature, summary in cluster_summaries.items():
    print(f"Summary statistics for {feature}:")
    display(summary)
    print("\n")


Summary statistics for Annual Income:


,mean,median,min,max,std,mode
Cluster,,,,,,
1,7.414173e+05,705000.0,13500,5046000,578913.951066,13500
2,6.385108e+05,663250.0,13500,1700000,454369.949803,13500
3,6.732915e+05,681000.0,10080,1870000,488860.092690,13500
4,2.331237e+06,2100000.0,1417250,11200000,796207.798688,1900000




Summary statistics for Price ($):


,mean,median,min,max,std,mode
Cluster,,,,,,
1,52308.411966,46000.0,36000,85800,12283.135044,45000
2,22110.672749,21001.0,1200,41001,6897.361016,19000
3,21089.275046,21000.0,1700,41001,6464.435367,22000
4,25252.009861,22500.0,1450,85000,9553.922624,22000




Summary statistics for Amount_paid_for_insurance:


,mean,median,min,max,std,mode
Cluster,,,,,,
1,2877.658761,2648.0,2070,4762,616.343484,2460
2,1351.398480,1331.0,450,2343,363.947216,1059
3,1302.220765,1277.0,187,2339,351.361269,996
4,1511.593008,1425.0,103,4520,496.577922,1311


Affluent Buyers:

Cluster 1 has the highest mean annual income and mean price, indicating a group of customers with high purchasing power.
These customers are likely to belong to a high-income bracket and are willing to spend on expensive products or services.
Budget-Conscious Consumers:

Cluster 2 comprises customers with lower mean annual income and lower mean price.
This group is characterized by their cautious spending habits and likely falls into lower-income categories.
Middle-Income Shoppers:

Cluster 3 represents customers with moderate mean annual income and moderate mean price.
These individuals have average spending habits and are likely to belong to middle-income households.
High-End Buyers:

Cluster 4 consists of customers with extremely high mean annual income.
Despite their smaller cluster size, these customers exhibit significantly higher spending compared to other clusters, indicating a segment of luxury or high-end buyers.

In [141]:
# Assigning names directly to clusters using iloc
df.loc[df['Cluster'] == 1, 'Cluster Name'] = "Affluent Buyers"
df.loc[df['Cluster'] == 2, 'Cluster Name'] = "Budget-Conscious Consumers"
df.loc[df['Cluster'] == 3, 'Cluster Name'] = "Middle-Income Shoppers"
df.loc[df['Cluster'] == 4, 'Cluster Name'] = "High-End Buyers"

# Display the updated DataFrame with cluster names
display(df.head())


,Quarter,Year,Gender,Annual Income,Dealer_Name,Company,Model,Engine,Transmission,Color,Price ($),Body Style,Amount_paid_for_insurance,Claim amount,City,Claim,Income_Bracket,Cluster,Cluster Name
0,1,2022,Female,671500,Scrivener Performance Engineering,Toyota,Sienna,Overhead Camshaft,Manual,Pale White,39501,Passenger,2182,0,Ventspils,0,High,1,Affluent Buyers
1,1,2022,Male,765000,Star Enterprises Inc,Lincoln,Navigator,Double Overhead Camshaft,Auto,Black,19001,Sedan,1280,0,Liepaja,0,High,2,Budget-Conscious Consumers
2,1,2022,Male,900000,Suburban Ford,Nissan,Pathfinder,Double Overhead Camshaft,Auto,Pale White,21001,SUV,1202,0,Ventspils,0,High,2,Budget-Conscious Consumers
3,1,2022,Female,820000,Tri-State Mack Inc,Mercury,Villager,Overhead Camshaft,Manual,Pale White,16000,Passenger,832,0,Ventspils,0,High,3,Middle-Income Shoppers
4,1,2022,Female,650000,U-Haul CO,Chrysler,Concorde,Double Overhead Camshaft,Auto,Black,12000,SUV,1046,0,Riga,0,High,2,Budget-Conscious Consumers


In [143]:
# Calculate count and percentage of insurance claims by each cluster
total_count = df.groupby('Cluster')['Claim'].count()
claims_count = df.groupby('Cluster')['Claim'].sum()
total_customers = df.groupby('Cluster')['Claim'].count()
claims_percentage = (claims_count / total_customers) * 100

# Calculate the percentage of customers who purchase insurance within each cluster
profit_per_person = (df.groupby('Cluster')['Amount_paid_for_insurance'].sum() - df.groupby('Cluster')['Claim amount'].sum()) / total_count

# Display the results
analysis_results = pd.DataFrame({
    'Cluster Name': ['Affluent Buyers', 'Budget-Conscious Consumers', 'Middle-Income Shoppers', 'High-End Buyers'],
    'Total': total_count,
    'Claims Count': claims_count.values,
    'Claims Percentage': claims_percentage.values,
    'Profit Per Person': profit_per_person.values
})

display(analysis_results)


,Cluster Name,Total,Claims Count,Claims Percentage,Profit Per Person
Cluster,,,,,
1,Affluent Buyers,4680,494,10.555556,2321.742094
2,Budget-Conscious Consumers,8550,825,9.649123,1140.916608
3,Middle-Income Shoppers,8235,835,10.139648,1092.365149
4,High-End Buyers,2231,213,9.547288,1274.579113
